In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# List to store the scraped data
data = []

# Base URL for page 1 (we will modify this URL for subsequent pages)
base_url = 'https://www.ft.com/search?expandRefinements=true&q=bitcoin&dateFrom=2024-04-01&dateTo=2024-09-23&page={}'

# Step 1: Loop through all 11 pages
for page_num in range(1, 12):  # Page numbers 1 to 11
    url = base_url.format(page_num)
    
    # Fetch the content from the current page
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.text, 'lxml')

        # Extract titles and dates
        articles = soup.select('div.o-teaser__content')  # Selects all articles

        for article in articles:
            # Extract the title
            title_tag = article.select_one('a.js-teaser-heading-link')
            title = title_tag.text.strip() if title_tag else 'No title'

            # Extract the date
            date_tag = article.select_one('time')
            date = date_tag['datetime'] if date_tag else 'No date'

            # Append the data as a tuple (Title, Date) to the list
            data.append((title, date))
    else:
        print(f"Failed to retrieve page {page_num}")

# Step 2: Convert the data into a DataFrame
df = pd.DataFrame(data, columns=['Title', 'Date'])

# Step 3: Save the DataFrame to a CSV file (optional)
df.to_csv('scraped_ft_articles.csv', index=False)

# Step 4: Display the DataFrame (or further manipulate it)
print(df)


Failed to retrieve page 10
Failed to retrieve page 11
                                                 Title  \
0    Trump’s call for a bitcoin strategic reserve i...   
1    Craig Wright said he invented bitcoin — lawyer...   
2                    Trump and the politics of bitcoin   
3    ‘The first crypto president’: bitcoin fans bet...   
4      Crypto miners hoard bitcoin as supply cut looms   
..                                                 ...   
220         Is it too late to invest in the gold rush?   
221                      The overstimulated superpower   
222                        A lifeline for Canary Wharf   
223  Markets update: Japanese equities rise followi...   
224  The Monaco hedge fund boss tied up with an ind...   

                         Date  
0    2024-09-02T04:00:03+0000  
1    2024-09-13T04:00:48+0000  
2    2024-07-31T12:11:32+0000  
3    2024-07-30T04:00:28+0000  
4    2024-04-18T04:00:01+0000  
..                        ...  
220  2024-04-26T13:36:14+

In [5]:
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get database credentials from environment variables
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')

# Create a connection string
connection_string = f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}'

# Create an SQLAlchemy engine
engine = create_engine(connection_string)

# Store DataFrame in the remote database
df.to_sql('articles', engine, if_exists='replace', index=False)


ModuleNotFoundError: No module named 'psycopg2'